In [1]:
from buttermilk import BM
import os
from hydra import initialize, compose
from omegaconf import OmegaConf


from promptflow.tracing import trace, start_trace
from rich import print as rprint

os.chdir('../..')
print(os.getcwd())

# Load config, specifying overrides for our particular job
with initialize(version_base=None, config_path="./conf"):
    cfg = compose(config_name='config',
                  overrides=["+flows=summarise_osb","job=export",])



# Load the main ButterMilk singleton instance
# This takes care of credentials, save paths, and other defaults
bm = BM(cfg=cfg)




/opt/homebrew/Caskroom/miniconda/base/envs/bm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/Users/suzor/src/buttermilk
2024-11-12 16:00:14 J5HW6L4KT6 buttermilk buttermilk.py[ 232] INFO {'message': "Logging setup for: 20241112T0600Z-4dT8-J5HW6L4KT6-suzor. Ready for data collection, saving log to Google Cloud Logs (Resource(type='generic_task', labels={'project_id': 'dmrc-platforms', 'location': 'us-central1', 'namespace': 'automod', 'job': 'export', 'task_id': '20241112T0600Z-4dT8-J5HW6L4KT6-suzor'})). Default save directory for data in this run is: gs://dmrc-analysis/runs/automod/export/20241112T0600Z-4dT8-J5HW6L4KT6-suzor", 'project': 'automod', 'job': 'export', 'run_id': '20241112T0600Z-4dT8-J5HW6L4KT6-suzor', 'save_dir': 'gs://dmrc-analysis/runs/automod/export/20241112T0600Z-4dT8-J5HW6L4KT6-suzor', 'ip': '159.196.210.27', 'node_name': 'J5HW6L4KT6', 'username': 'suzor'}
2024-11-12 16:00:14 J5HW6L4KT6 buttermilk buttermilk.py[ 240] DEBUG Buttermilk version is: 0.2.0
Prompt flow service has started...


{
    'name': 'automod',
    'job': 'export',
    'logger': {'type': 'gcp'},
    'tracing': {'enabled': True},
    'verbose': True,
    'save_dest': {'type': 'gcp', 'project': 'dmrc-analysis', 'region': 'us-central1', 'bucket': 'dmrc-analysis'},
    'secret_provider': {
        'type': 'azure',
        'vault': 'https://suzorvault.vault.azure.net/',
        'resource_group': 'rg-suzor_ai',
        'models_secret': 'models'
    },
    'run': {'platform': 'local', 'max_concurrency': 32},
    'flows': [
        {
            'name': 'summarise OSB cases',
            'num_runs': 3,
            'concurrency': 20,
            'agent': {
                'type': 'LC',
                'template': 'summarise',
                'instructions': 'summarise_osb',
                'model': ['gpt4o', 'sonnet', 'gemini15pro']
            },
            'parameters': {'record': 'record'},
            'data': [
                {
                    'name': 'osb all cases',
                    'type': 'file',
                    'uri': 'gs://dmrc-platforms/data/osb/cases.json',
                    'columns': {'record_id': 'record_id', 'text': 'text'}
                }
            ],
            'save': {
                'type': 'bq',
                'dataset': 'dmrc-analysis.toxicity.flow',
                'schema': 'buttermilk/schemas/flow.json'
            }
        },
        {
            'name': 'synthesise OSB cases',
            'num_runs': 1,
            'concurrency': 20,
            'agent': {
                'type': 'LC',
                'template': 'synthesise',
                'instructions': 'summarise_osb',
                'model': ['gpt4o', 'sonnet']
            },
            'parameters': {'content': 'text', 'answers': 'answers'},
            'data': [
                {
                    'name': 'osb all cases',
                    'type': 'file',
                    'uri': 'gs://dmrc-platforms/data/osb/cases.json',
                    'columns': {'record_id': 'record_id', 'text': 'text'}
                },
                {
                    'name': 'synthesise previous runs',
                    'type': 'job',
                    'dataset': 'dmrc-analysis.toxicity.flow',
                    'max_records_per_group': 4,
                    'last_n_days': 1,
                    'agg': True,
                    'filter': {
                        'agent_info.flow': 'summarise OSB cases',
                        'parameters.model': ['gpt4o', 'sonnet', 'gemini15pro']
                    },
                    'join': {'record_id': 'record.record_id'},
                    'group': {
                        'flow': 'agent_info.flow',
                        'template': 'agent_info.template',
                        'model': 'parameters.model'
                    },
                    'columns': {
                        'answers': {'id': 'job_id', 'text': 'outputs'},
                        'meta': {
                            'flow': 'agent_info.flow',
                            'template': 'agent_info.template',
                            'agent_info': 'agent_info',
                            'model': 'parameters.model',
                            'timestamp': 'timestamp'
                        }
                    }
                }
            ],
            'save': {
                'type': 'bq',
                'dataset': 'dmrc-analysis.toxicity.flow',
                'schema': 'buttermilk/schemas/flow.json'
            }
        },
        {
            'name': 'export',
            'agent': {'type': 'GSheetExporter', 'sheet_name': 'OSB summaries'},
            'save': {'type': 'gsheets', 'destination': 'new'},
            'data': [
                {
                    'name': 'osb all cases',
                    'type': 'file',
                    'uri': 'gs://dmrc-platforms/data/osb/cases.json',
                    'columns': {'record_id': 'record_id', 'text': 'text'}
                },
                {
        

I0000 00:00:1731391214.108159 63585506 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


2024-11-12 16:00:16 J5HW6L4KT6 buttermilk save.py[ 272] DEBUG Uploading file gs://dmrc-analysis/runs/automod/export/20241112T0600Z-4dT8-J5HW6L4KT6-suzor/config_UiZYez.json.


DEBUG:buttermilk:Uploading file gs://dmrc-analysis/runs/automod/export/20241112T0600Z-4dT8-J5HW6L4KT6-suzor/config_UiZYez.json.


2024-11-12 16:00:17 J5HW6L4KT6 buttermilk save.py[ 275] DEBUG Successfully uploaded file gs://dmrc-analysis/runs/automod/export/20241112T0600Z-4dT8-J5HW6L4KT6-suzor/config_UiZYez.json with 6216 lines written.


DEBUG:buttermilk:Successfully uploaded file gs://dmrc-analysis/runs/automod/export/20241112T0600Z-4dT8-J5HW6L4KT6-suzor/config_UiZYez.json with 6216 lines written.


In [2]:
from buttermilk.runner.helpers import load_data
from buttermilk.utils.gsheet import GSheet

from buttermilk.runner.orchestrator import MultiFlowOrchestrator
orchestrator = MultiFlowOrchestrator(flow=cfg.flows[2], source="notebook")

await orchestrator.prepare_data()

df = orchestrator._dataset
df


2024-11-12 16:00:30 J5HW6L4KT6 buttermilk buttermilk.py[ 312] INFO Query stats: Ran in 0:00:02.745665 seconds, cache hit: None, billed None, approx cost $unknown.


INFO:buttermilk:Query stats: Ran in 0:00:02.745665 seconds, cache hit: None, billed None, approx cost $unknown.


,record_id,text,model,timestamp,title,date,content,reasons,recommendations,result,type,topics,standards,location,outputs
45,FB-LXNFAD5F,"{""result"": ""upheld"", ""title"": ""Haitian Police ...",sonnet,2024-11-12 05:52:17.214743+00:00,Haitian Police Station Video,2023-05,A video shows civilians entering a police stat...,[The Violence and Incitement Community Standar...,[Meta should assess the effectiveness and time...,leave up,standard,"[Freedom of expression, Safety, Violence, Cris...",[Violence and Incitement Community Standard],Haiti,{'content': 'A video shows civilians entering ...
43,FB-TTXIBH8S,"{""result"": ""upheld"", ""title"": ""Fictional Assau...",gpt4o,2024-11-12 05:51:34.081520+00:00,Fictional Assault on Gay Couple,2023-07,A 30-second scripted video showed actors depic...,[The Hate Speech policy prohibits calls to act...,[Clarify that implicit references to protected...,takedown,summary,"[Discrimination, LGBT, Violence]",[Hate Speech],United Kingdom,{'content': 'A 30-second scripted video showed...
32,FB-6YHRXHZR,"{""result"": ""upheld"", ""title"": ""Pro-Navalny pro...",gpt4o,2024-11-12 02:07:03.383961+00:00,Pro-Navalny protests in Russia,2021-01-24,A user ('the Protester') participated in the p...,[The Bullying and Harassment Community Standar...,[Clarify the relationship between the rational...,leave up,standard,"[Freedom of expression, Politics, Russia, Prot...",[Bullying and Harassment Community Standard],Russia,{'content': 'A user ('the Protester') particip...
5,IG-WUC3649N,"{""result"": ""upheld"", ""title"": ""Al-Shifa Hospit...",gpt4o,2024-11-12 05:51:38.684778+00:00,Al-Shifa Hospital,2023-12-19,A video posted on Instagram showing the afterm...,[The Violent and Graphic Content Community Sta...,[Amend the Violent and Graphic Content Communi...,leave up,expedited,"[Safety, Violence, War and conflict, Humanitar...",[Violent and Graphic Content Community Standard],Gaza,{'content': 'A video posted on Instagram showi...
28,FB-CZHY85JC,"{""result"": ""upheld"", ""title"": ""Sri Lanka pharm...",sonnet,2024-11-12 02:07:58.885103+00:00,Sri Lanka pharmaceuticals,2022-04-27,A medical trade union in Sri Lanka posted an i...,[The Restricted Goods and Services Community S...,[Publish information about the 'spirit of the ...,leave up,standard,"[Healthcare, Pharmaceuticals, Crisis Response,...",[Restricted Goods and Services Community Stand...,Sri Lanka,{'content': 'A medical trade union in Sri Lank...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,FB-IZP492PJ,"{""result"": ""upheld"", ""title"": ""Corruption of l...",gpt4o,2024-11-12 05:51:40.412627+00:00,Corruption of law enforcement in Indonesia,2023-04-01,A user posted a video monologue in Bahasa Indo...,[The Violence and Incitement policy prohibits ...,[Content moderation systems should be designed...,leave up,summary,"[Government, Corruption, Law Enforcement, Poli...",[Violence and Incitement],Indonesia,{'content': 'A user posted a video monologue i...
49,FB-I964KKM6,"{""result"": ""upheld"", ""title"": ""Colombian polic...",sonnet,2024-11-12 02:07:56.399236+00:00,Colombian police cartoon,2022-01,A cartoon resembling the official crest of the...,[The Dangerous Individuals and Organizations p...,[Ensure content with high rates of successful ...,leave up,standard,"[Police violence, Political speech, Protest, A...","[Dangerous Individuals and Organizations, Viol...",Colombia,{'content': 'A cartoon resembling the official...
40,FB-TYE2766G,"{""result"": ""upheld"", ""title"": ""South Africa sl...",gpt4o,2024-11-12 05:51:29.697797+00:00,South Africa slurs,2021-05,A user posted in English in a public Facebook ...,[Tier 3 of the Hate Speech Community Standard ...,[Facebook should notify users about the specif...,takedown,standard,"[Race, Slurs, Hate Speech, South Africa]",[Hate Speech Community Standard - Tier 3 (slurs)],South Africa,{'content': 'A user posted in English in a pub...
18,FB-E5M6QZGA,"{""result"": ""upheld"", ""title"": ""Colombia protes..

In [3]:
# save the input data from this step to a spreadsheet so that we can compare later.
from buttermilk.utils.gsheet import GSheet, format_strings

answers = format_strings(orchestrator._dataset, convert_json_columns=['outputs'])

gsheet = GSheet()
sheet = gsheet.save_gsheet(df=answers,  sheet_name="syntheses", title="OSB Summaries")
sheet_id = sheet.id



2024-11-12 16:01:40 J5HW6L4KT6 buttermilk gsheet.py[ 118] INFO Saved to https://docs.google.com/spreadsheets/d/1O3ffemi_it64qko82FjGKAC-w8y_W8zV46-n5ojWsaU


INFO:buttermilk:Saved to https://docs.google.com/spreadsheets/d/1O3ffemi_it64qko82FjGKAC-w8y_W8zV46-n5ojWsaU


In [ ]:
# rotate the output around and save it to a new sheet

import pandas as pd
import json
orchestrator._dataset.output_backup
# evals = prepare_step_data(cfg.data, cfg.step[3].data)
# evals = evals.set_index(["record_id","model"]).sort_values("timestamp", ascending=False).reset_index().drop_duplicates(["record_id", "model"])
# evals = format_strings(evals, convert_json_columns=['analysis'])
# evals = evals.set_index(["record_id"]).pivot(columns="model",values="analysis")
# sheet = gsheet.save_gsheet(df=evals, sheet_id=sheet_id, sheet_name="evals", title="results")



In [ ]:
# save the cases df to GCS
# df[['record_id', 'text']].drop_duplicates().reset_index(drop=True).to_json(orient='records', lines=True, path_or_buf='gs://dmrc-platforms/data/osb/cases.json')
